In [1]:
import s3fs
import os
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
from tqdm import tqdm

s3 = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'},
    key=os.environ["AWS_ACCESS_KEY_ID"],
    secret=os.environ["AWS_SECRET_ACCESS_KEY"],
    token=os.environ["AWS_SESSION_TOKEN"]
)

In [2]:
root = "leoacpr/diffusion/"
prefix = "UKCHLL"

ids = []

for path in s3.ls(root):
    folder = path.split(root)[1]
    if prefix in folder: ids.append(folder)

ids

['UKCHLL001',
 'UKCHLL002',
 'UKCHLL009',
 'UKCHLL011',
 'UKCHLL015',
 'UKCHLL017',
 'UKCHLL021',
 'UKCHLL023',
 'UKCHLL031',
 'UKCHLL034',
 'UKCHLL035',
 'UKCHLL037',
 'UKCHLL038',
 'UKCHLL039',
 'UKCHLL041',
 'UKCHLL042',
 'UKCHLL043',
 'UKCHLL045',
 'UKCHLL046',
 'UKCHLL048']

In [8]:
if not os.path.exists("nnUNet_raw"): os.mkdir("nnUNet_raw")
if not os.path.exists("nnUNet_preprocessed"): os.mkdir("nnUNet_preprocessed")
if not os.path.exists("nnUNet_results"): os.mkdir("nnUNet_results")
    
if not os.path.exists("nnUNet_raw/Dataset475_CURVAS"): os.mkdir("nnUNet_raw/Dataset475_CURVAS")
if not os.path.exists("nnUNet_raw/Dataset475_CURVAS/labelsTr"): os.mkdir("nnUNet_raw/Dataset475_CURVAS/labelsTr")
if not os.path.exists("nnUNet_raw/Dataset475_CURVAS/imagesTr"): os.mkdir("nnUNet_raw/Dataset475_CURVAS/imagesTr")

In [9]:
generate_dataset_json(
    output_folder = "nnUNet_raw/Dataset475_CURVAS",
    channel_names = {0: "CT"},
    labels = {
        "background": 0,
        "pancreas": 1,
        "kidney": 2,
        "liver": 3
    },
    num_training_cases = len(ids),
    file_ending = ".nii.gz",
    dataset_name = "CURVAS"
)

In [22]:
for i in tqdm(ids, desc="Téléchargement des images"):
    s3.get(root + i + "/image.nii.gz", "nnUNet_raw/Dataset475_CURVAS/imagesTr/CURVAS_" + i + "_0000.nii.gz")

Téléchargement des images: 100%|██████████| 20/20 [00:52<00:00,  2.64s/it]


In [23]:
for i in tqdm(ids, desc="Téléchargement de la première anotation"):
    s3.get(root + i + "/annotation_1.nii.gz", "nnUNet_raw/Dataset475_CURVAS/labelsTr/CURVAS_" + i + ".nii.gz")

Téléchargement de la première anotation: 100%|██████████| 20/20 [00:02<00:00,  9.93it/s]


In [5]:
os.environ["test"] = os.path.abspath("nnUNet_raw")

In [6]:
os.environ["test"]

'/home/onyxia/work/statapp_2025_curvas/nnUNet_raw'